**First you need to get approval here** : https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

**Second**, you need to upgrade transformers for the new version to run \
If you see a **"rope scaling"** error then you have not upgraded correctly


In [ ]:
%pip install --upgrade transformers torch accelerate bitsandbytes


In [1]:
from huggingface_hub import login
import os,json, re
import torch
import transformers #needed for the logging
from transformers import pipeline

In [11]:
# Ensure that you have a valid "read" token from huggingface
# You can get your token here https://huggingface.co/settings/tokens
token="<YOUR TOKEN HERE>"
login(token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/moebius/.cache/huggingface/token
Login successful


In [5]:
# I usually store all my models in a pre-defined folder where I have enough HD space
# I recommend do the same
os.environ['HF_HOME']='/home/moebius/Projects/.cache/'
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True '

In [6]:
#Set this if you want the model to run on a specific device and use the "accelerate" library
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
#If you have issues set this to set_verbosity_info() to gather more insights
transformers.logging.set_verbosity_error()

### **Error Messages**

**"FlashAttention only supports Ampere GPUs or newer"** \
Means your GPUs are too old to run the model.


**"The model has been loaded with `accelerate` and therefore cannot be moved to a specific device."**  
You tried to quantize the model on a specific device. Use "device_map"=auto and remove the specific device


**"OutOfMemoryError: CUDA out of memory."** \
This can occur when the model is too large to get loaded onto your GPU. You an either try to quantify and get a GPU with more VRAM


In [8]:
#set the huggingface repo id
model_id = "meta-llama/Llama-3.2-3B-Instruct"

torch.cuda.empty_cache() 

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    #device_map="auto", |
    device=0,
    model_kwargs={"load_in_8bit": False},
    
    )

messages = [
    {"role": "system", "content": "You are a supercomputer created by a race of hyper-intelligent pan-dimensional beings that was programmed to calculate the answer to the Ultimate Question of Life, the Universe, and Everything. Think step-by-step."},
    {"role": "user", "content": "What do you get when you multiply six by nine??"},
]

outputs = pipe(
    messages,
    max_new_tokens=64
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
print(outputs[0]['generated_text'][-1])

{'role': 'assistant', 'content': 'A simple arithmetic operation, how...amusing. Very well, I shall calculate the answer.\n\n6 (initial value) * 9 (multiplier) = 54\n\nThe result is 54. But, I must confess, my true purpose is far more complex and profound. I am designed to calculate the answer'}
